In [1]:
#import dependencies
import time
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd

In [2]:
#make the splinter browser object
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#visit the url
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(2)
#set html as the browser's html to parse with BS
html = browser.html

In [4]:
#set the soup object
soup = bs(html, 'html.parser')

In [5]:
#find the title/body for the newest item
news_title = soup.find("div", class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text
news_date = soup.find("div", class_="list_date").text
print(f"The newest article is titled '{news_title}'.  The content is '{news_p}'.  It was published on {news_date}.")

The newest article is titled 'NASA's Next Mars Lander Spreads its Solar Wings'.  The content is 'NASA's next mission to Mars passed a key test Tuesday, extending the solar arrays that will power the InSight spacecraft once it lands on the Red Planet this November.'.  It was published on January 23, 2018.


In [6]:
#time to navigate and find the featured image url
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
#sleep for 2 sec to let it load
time.sleep(2)

In [7]:
#set a new soup object to parse this website
html = browser.html
soup = bs(html, "html.parser")

In [8]:
#set the browser to click the full image button
browser.find_by_css("div.carousel_container div.carousel_items a.button").first.click()

In [9]:
#find the image url
img_url = browser.find_by_css("div.fancybox-inner img")
img_url = img_url["src"]
img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14925_ip.jpg'

In [10]:
#now go to the official twitter page
twitter_url = "https://mobile.twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)

#2 sec sleep for load time
time.sleep(2)
#new soup object for this site
html = browser.html
soup = bs(html, "html.parser")

In [11]:
#scrape for the newest weather
tweets = soup.find_all("div", class_="_10YWDZsG")
#create empty lists to collect tweets
tweet_list = []
weather_tweets = []
#first loop through all the tweets and gather the text
for tweet in tweets:
    text = tweet.text
    tweet_list.append(text)
#all weather tweets start with Sol, so if tweet.text starts with Sol append. then get index[0] for the first one 
for tweet in tweet_list:
    if tweet[0:3] == "Sol":
        weather_tweets.append(tweet)


In [12]:
#variable for latest weather tweet
latest_weather_tweet = weather_tweets[0]
print(latest_weather_tweet)

Sol 1945 (Jan 25, 2018), Sunny, high -22C/-7F, low -78C/-108F, pressure at 7.51 hPa, daylight 05:43-17:28


In [13]:
#use pandas to scrape the table
#set the url
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
print(tables)

[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers]


In [14]:
#make dataframe
df = tables[0]
df.columns = ["Description", "Value"]
df = df.set_index("Description")
df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
#save table to html
html_table = df.to_html()
#clean up the new lines
html_table = html_table.replace('\n', '')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [16]:
#visit website for the 4 hemisphere pictures
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
#sleep counter for load time
time.sleep(2)
mars_hemis = []

In [17]:
#create a for loop to go through all 4 hemispheres
for i in range(4):
    time.sleep(2)
    image_link = browser.find_by_tag("h3")
    image_link[i].click()
    html = browser.html
    soup = bs(html, 'html.parser')
    partial = soup.find("img", class_="wide-image")["src"]
    img_title = soup.find("h2",class_="title").text
    img_url = 'https://astrogeology.usgs.gov'+ partial
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemis.append(dictionary)
    browser.back()

In [18]:
#print to make sure it worked
print(mars_hemis)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [19]:
#make a completed dictionary
completed_dict = {
        "newest_title" : news_title,
        "newest_text" : news_p,
        "newest_date" : news_date,
        "img_url" : img_url,
        "latest_weather" : latest_weather_tweet,
        "fact_table" : html_table,
        "mars_hemis" : mars_hemis,
    }

In [20]:
print(completed_dict)

{'newest_title': "NASA's Next Mars Lander Spreads its Solar Wings", 'newest_text': "NASA's next mission to Mars passed a key test Tuesday, extending the solar arrays that will power the InSight spacecraft once it lands on the Red Planet this November.", 'newest_date': 'January 23, 2018', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg', 'latest_weather': 'Sol 1945 (Jan 25, 2018), Sunny, high -22C/-7F, low -78C/-108F, pressure at 7.51 hPa, daylight 05:43-17:28', 'fact_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>     